In [ ]:
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)

import pandas as pd
import numpy as np 
from functools import partial
import torch.nn as nn
import torch 
from codes.models.data_form.DataForm import DataTransfo_1SNP, PatientList
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


In [ ]:
    labels_1 = labels[data[:,pheno_nb]==1]
    labels_0 = labels[data[:,pheno_nb]==0]
    P0 = np.sum(labels_0==1)/len(labels_0)
    P1 = np.sum(labels_1==1)/len(labels_1)
    F0 = max(P0, 1-P0)
    F1 = max(P1, 1-P1)
    return F0, F1

In [ ]:
### data constants:
CHR = 1
SNP = 'rs673604'
pheno_method = 'Abby' # Paul, Abby
rollup_depth = 4
Classes_nb = 2 #nb of classes related to an SNP (here 0 or 1)
vocab_size = None # to be defined with data
padding_token = 0
prop_train_test = 0.8
load_data = False
save_data = False
remove_none = True
decorelate = True
equalize_label = False
threshold_corr = 0.9
threshold_rare = 50
remove_rare = 'all' # None, 'all', 'one_class'
compute_features = True
padding = False
list_env_features = ['age', 'sex']
### data format
batch_size = 20
data_share = 1/10000

In [ ]:
dataT = DataTransfo_1SNP(SNP=SNP,
                         CHR=CHR,
                         method=pheno_method,
                         padding=padding,  
                         pad_token=padding_token, 
                         load_data=load_data, 
                         save_data=save_data, 
                         compute_features=compute_features,
                         prop_train_test=prop_train_test,
                         remove_none=True,
                         equalize_label=equalize_label,
                         rollup_depth=rollup_depth,
                         decorelate=decorelate,
                         threshold_corr=threshold_corr,
                         threshold_rare=threshold_rare,
                         remove_rare=remove_rare, 
                         list_env_features=list_env_features,
                         data_share=data_share)
#patient_list = dataT.get_patientlist()


In [ ]:
data, labels, indices_env, name_envs = dataT.get_tree_data(with_env=False)
data, labels = DataTransfo_1SNP.equalize_label(data, labels)
nb_phenos = data.shape[1]


In [ ]:
#### decorelate the dataTree
np.sum(data.var(axis=0)==0)

In [ ]:
phenos = np.arange(nb_phenos)

def get_risk_pheno(data, labels, pheno_nb):
    labels_ac = labels[data[:,pheno_nb]==1]
    labels_deac = labels[data[:,pheno_nb]==0]
    proba_mut_ac = np.sum(labels_ac==1)/len(labels_ac)
    proba_mut_deac = np.sum(labels_deac==1)/len(labels_deac)
    ratio  = proba_mut_ac / proba_mut_deac
    return ratio
def get_pred_naive(data, labels, pheno_nb, proba=False):
    labels_ac = labels[data[:,pheno_nb]==1]
    nb_ones_ac = np.sum(labels_ac==1)
    nb_zeros_ac = np.sum(labels_ac==0)
    proba = nb_zeros_ac / len(labels_ac)
    label = (1 if nb_ones_ac > nb_zeros_ac else 0)
    return proba, label
get_risk_pheno = partial(get_risk_pheno, data, labels)
get_pred_naive = partial(get_pred_naive, data, labels)

In [ ]:
odds_ratios = list(map(get_risk_pheno, phenos))
pred_naive = np.array(list(map(get_pred_naive, phenos)))
probas_pred_naive = pred_naive[:, 0]
labels_pred_naive = pred_naive[:, 1]

def get_pred_sentence(probas_pred_naive, labels_pred_naive, sentence, method='max'):
    sentence = sentence.astype(bool)
    labels_naive = labels_pred_naive[sentence].astype(bool)
    probas_naive = probas_pred_naive[sentence].astype(bool)

    if method=='mean':
        if np.mean(probas_naive)>0.5:
            return 1
        else:
            return 0
    if method=='max':
        argmax = np.argmax((probas_naive-0.5)**2)
        return labels_naive[argmax]
    
#get_pred_sentence = partial(get_pred_sentence, probas_pred_naive, labels_pred_naive)

In [ ]:
mask = 1 - np.isnan(probas_pred_naive)
data = data[:, mask]
probas_pred_naive = probas_pred_naive[mask]
labels_pred_naive = labels_pred_naive[mask]


In [ ]:
get_pred_sentence = partial(get_pred_sentence, probas_pred_naive, labels_pred_naive)

In [ ]:
frequencies = np.sum(data, axis=0)
labels_pred  = np.apply_along_axis(get_pred_sentence, arr=data, axis=1)
np.sum(labels_pred == labels)/len(labels)

In [ ]:
data.shape

In [ ]:
len(probas_pred_naive[np.sqrt((probas_pred_naive-0.5)**2)>0.2])

In [ ]:
max(frequencies[np.sqrt((probas_pred_naive-0.5)**2)>0.1])

In [ ]:
np.sum(probas_pred_naive==0.5)

In [ ]:
probas_pred_naive.notnan()

In [ ]:
frequencies[np.argwhere(np.isnan(probas_pred_naive))]